In [1]:
%matplotlib inline

import sys
import os
import numpy as np
import json
import pandas as pd
import traceback

sys.path.append("../../../") # add results folder as a python path 
from viz import *
from analyze import *

In [2]:
path = os.path.dirname("./data2/BOHB/")
target_dirs = os.listdir(path)

In [3]:
def getTrialResult(result):
    cum_exec_time = []
    cum_opt_time = []
    cum_first = True

    exec_times = []
    opt_times = []
    epochs = []
    model_info = []
    errors = []
    accuracies = []

    for r in result:
        dict_r = json.loads(r)
        exec_time = dict_r[3]['info']['elapsed time']
        opt_time = dict_r[2]['finished'] - dict_r[2]['submitted']

        if cum_first is True:
            cum_exec_time_last_value = 0.0
            cum_opt_time_last_value = 0.0
            cum_first = False

        if cum_exec_time_last_value <= 86400:
            exec_times.append(exec_time)
            opt_times.append(opt_time)
            cum_exec_time.append((cum_exec_time_last_value + exec_time))
            cum_opt_time.append((cum_opt_time_last_value + opt_time))
            epochs.append(dict_r[1])
            model_info.append(dict_r[0])
            errors.append(dict_r[3]['loss'])
            accuracies.append(dict_r[3]['info']['test accuracy'])

            cum_exec_time_last_value = cum_exec_time[len(cum_exec_time)-1]
            cum_opt_time_last_value = cum_opt_time[len(cum_opt_time)-1]
            
            dict_result = dict([("cum_exec_time", cum_exec_time),
                        ("exec_time", exec_times),
                        ("opt_time", opt_times),
                        ("model_info", model_info),
                        ("cum_opt_time", cum_opt_time),
                        ("error", errors),
                        ("accuracy", accuracies),
                        ("epoch", epochs)])
        else:
            continue
        
    return dict_result

In [4]:
def getTrialConfig(config):
    config_dict = dict()
    
    for c in config:
        dict_c = json.loads(c)
        target = dict_c[2]
        config_dict[str(dict_c[0])] = target
        
    return config_dict

In [5]:
def addModelInfo(trial_result, trial_config):
    model_index_list = []
    for i in range(len(trial_result['model_info'])):
        model_idx = str(trial_result['model_info'][i])
        model_info = trial_config[model_idx]
        target_index = -2
        if 'lookup_index' in model_info:
            target_index = model_info['lookup_index']
        model_index_list.append(target_index)
    trial_result['model_index'] = model_index_list
    del trial_result['model_info']

In [6]:
bohb_results = dict()
bohb_configs = dict()

for d in target_dirs:
    trial_num = d[5:]
    
    with open(path+'/'+d+'/'+'results.json') as rf:
        result = rf.readlines()

    result = [x.strip() for x in result]
    result_dict=getTrialResult(result)
    
    with open(path+'/'+d+'/'+'configs.json') as rf:
        config = rf.readlines()
        
    config = [x.strip() for x in config]
#     bohb_configs[trial_num]=getTrialConfig(config)
    config_dict=getTrialConfig(config)
    
    addModelInfo(result_dict, config_dict)
    
    bohb_results[trial_num]=result_dict
    

In [7]:
with open('data2_result.json', 'w') as outfile:
    json.dump(bohb_results, outfile)

In [8]:
bohb_results["0"]

{'cum_exec_time': [35.97000334444444,
  65.78933167777777,
  84.60396777777777,
  112.73652533333332,
  198.97102009999998,
  228.48232021111107,
  263.44815898888885,
  300.1481805444444,
  337.4672413222222,
  421.8649139888889,
  680.5683982888888,
  785.4659146222222,
  1038.6589326222222,
  1100.134670988889,
  1184.8167510222222,
  1438.9843609555555,
  1623.4115760555555,
  1950.4127791555557,
  2296.8492062555556,
  2554.1421673555556,
  2613.0735354666667,
  2632.207135466667,
  2685.4305300222222,
  2732.5892555777777,
  2819.0351706888887,
  2893.9633430111107,
  2920.549982333333,
  2957.749173555555,
  2969.5029360111107,
  3194.2874529777773,
  3305.8850266444438,
  3341.1463140111105,
  4015.4998649111103,
  4064.00082861111,
  4227.434870577777,
  4364.63069831111,
  4510.13358941111,
  5225.79894131111,
  5401.47783941111,
  5636.20846931111,
  5714.289729966666,
  5750.67927321111,
  5781.025977322221,
  5832.6339180999985,
  5882.1056217666655,
  5918.1751868777765,
